In [1]:
import numpy as np
import re
import pandas as pd

In [2]:
from collections import Counter

In [3]:
texts = pd.read_csv('data/labeled.csv')

In [4]:
texts.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [5]:
texts = texts.comment.values

In [6]:
texts = ''.join(i for i in texts)

In [7]:
texts[:100]

'Верблюдов-то за что? Дебилы, бл...\nХохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов е'

In [8]:
def pad(x, char, n=1, left=True):
    if left:
        return ' '.join(char for i in range(n)) +" " + x
    else:
        return x+" "+ ' '.join(char for i in range(n)) 
    
def process_data(x, start_token='<start>', end_token='<end>', lower=False, n_gram=2):
    n_gram = max(n_gram,2)
    if isinstance(x, str):
        x = re.split(r'[.?!]\s*',x)
    x = [re.sub('[^0-9a-zA-ZА-я]+', ' ', sentence).strip() for sentence in x]
    if lower:
        x = [sentence.lower() for sentence in x]
    x = [sentence.replace('.','').replace(',','').replace('!','').replace('?','').strip() for sentence in x if sentence]
    x = [pad(sentence, start_token, n_gram-1, left=True) for sentence in x if sentence]
    x = [pad(sentence, end_token, 1, left=False) for sentence in x if sentence]
    return x

In [9]:
processed_texts = process_data(texts, lower=True, n_gram=4)

In [10]:
processed_texts [:10]

['<start> <start> <start> верблюдов то за что <end>',
 '<start> <start> <start> дебилы бл <end>',
 '<start> <start> <start> хохлы это отдушина затюканого россиянина мол вон а у хохлов еще хуже <end>',
 '<start> <start> <start> если бы хохлов не было кисель их бы придумал <end>',
 '<start> <start> <start> собаке собачья смерть страницу обнови дебил <end>',
 '<start> <start> <start> это тоже не оскорбление а доказанный факт не дебил про себя во множественном числе писать не будет <end>',
 '<start> <start> <start> или мы в тебя верим это ты и твои воображаемые друзья <end>',
 '<start> <start> <start> тебя не убедил 6 страничный пдф в том что скрипалей отравила россия <end>',
 '<start> <start> <start> анализировать и думать пытаешься <end>',
 '<start> <start> <start> ватник что ли <end>']

In [11]:
from tqdm import tqdm

In [12]:
from collections import defaultdict

In [13]:
class MarkovChains:
    def __init__(self, n_grams=2, texts=None):
        self.n_grams = n_grams
        self.counter_dict = None
        self.n_tokens = None
        self.vocab = None
        self.inverse_vocab = None
        self.texts = texts
        if not self.texts is None:
            self.counter_dict, self.vocab, self.n_tokens = self.build_vocab()
        self.dict_probs = defaultdict(dict)
        
    def build_vocab(self):
        if isinstance(self.texts, list):
            vocab = list(set(np.hstack([i.split(' ') for i in self.texts])))
        elif isinstance(str): 
            vocab = list(set(self.texts.split(' ')))
            self.texts = [pad(i.strip(), '<end>', left=False) for i in self.texts.split('<end>')]
        counter_dict = Counter([])
        for sentence in tqdm(processed_texts):
            tokens = sentence.split(' ')
            for l in range(len(tokens)-self.n_grams+2):
                counter_dict.update([' '.join(tokens[l:l+self.n_grams-1])])
        n_tokens = len(vocab)
        print('Build vocabulary')
        return counter_dict, vocab, n_tokens
        
    def train(self, texts=None):
        if texts is None:
            assert self.texts, 'Please provide texts data'
        else:
            self.texts = texts
        if self.vocab is None:
            self.counter_dict, self.vocab, self.n_tokens = self.build_vocab()
        n_gram_counter = Counter([])
        for sentence in tqdm(processed_texts):
            tokens = sentence.split(' ')
            for l in range(len(tokens)-self.n_grams+1):
                n_gram_counter.update([' '.join(tokens[l:l+self.n_grams])])
        for n_gram, freq in tqdm(n_gram_counter.items()):
            tokens = n_gram.split(' ')
            main_token = tokens[-1]
            cond_tokens = ' '.join(tokens[:-1])
            cond_tokens_freq = self.counter_dict[cond_tokens]
            self.dict_probs[cond_tokens][main_token] = freq/cond_tokens_freq
        print('Finished training')
    
    def generate(self, start_token='<start>', end_token='<end>', greedy=False):
        start_token = [start_token for i in range(self.n_grams-1)]
        generated_tokens = [' '.join(start_token)]
        next_token = start_token
        while True:
            generated_tokens = list(np.hstack(i.split(' ') for i in generated_tokens))
            probs = self.dict_probs[' '.join(generated_tokens[-self.n_grams+1:])] 
            variants = list(probs.keys())
            probs = list(probs.values())
            if greedy:
                idx = np.argmax(probs)
            else:
                idx = np.random.choice(range(len(variants)), p=probs)
            next_token = variants[idx]
            if next_token==end_token:
                break
            generated_tokens.append(next_token)
        generated_tokens.append('.')
        generation = ' '.join(generated_tokens[self.n_grams:]).capitalize()
        return generation
    
    def get_params(self):
        return self.dict_probs

In [15]:
model = MarkovChains(n_grams=4, texts=processed_texts)

100%|██████████| 35036/35036 [00:01<00:00, 22885.58it/s]

Build vocabulary


In [16]:
model.train()

100%|██████████| 389341/389341 [00:01<00:00, 330697.13it/s]

Finished training


In [33]:
model.generate()

/Users/volodymyrkovenko/envs/plexai/lib/python3.7/site-packages/ipykernel_launcher.py:53: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


'Ключ переломлен пополам а наш батюшка женя совсем усох он разложился на пряник и на липовый яд поросвещенье вс ид т по плану да кремлеботов как урукхаев из грязи и говна делают .'